In [20]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score

%matplotlib inline

In [26]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
y = iris.target

X.shape

(150, 4)

#### 1. Обучить любую модель классификации на датасете iris до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [32]:
X_pca = PCA(n_components=2, random_state=42).fit_transform(X)
X_pca.shape

(150, 2)

In [29]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.25)

In [30]:
clf = RFC(n_jobs=-1).fit(X_train, y_train)
clf_pca = RFC(n_jobs=-1).fit(X_train_pca, y_train_pca)

In [31]:
print(f"До применения PCA F1-score: {f1_score(y_test, clf.predict(X_test), average='micro')}")
print(f"После применения PCA F1-score: {f1_score(y_test_pca, clf_pca.predict(X_test_pca), average='micro')}")

До применения PCA F1-score: 0.9736842105263158
После применения PCA F1-score: 0.9736842105263158


#### 2. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [15]:
means = X.mean(axis=0)
stds = X.std(axis=0, ddof=1)

X_scaled = (X - means)/stds

In [16]:
def fro_norm(X):
    return np.sum(X ** 2) ** 0.5

In [17]:
def pca_with_svd(X, n_components):
    U, s, Vt = np.linalg.svd(X)
    
    D = np.zeros_like(X, dtype=float)
    D[np.diag_indices(min(X.shape))] = s
    V = Vt.T
    
    # проверка
    np.testing.assert_array_almost_equal(U.dot(D).dot(Vt), X)
    
    W = V[:, :n_components]
    Z = X.dot(W)
    
    return Z

In [18]:
X_pca_4 = pca_with_svd(X_scaled, n_components=4)
X_pca_3 = pca_with_svd(X_scaled, n_components=3)
X_pca_2 = pca_with_svd(X_scaled, n_components=2)
X_pca_1 = pca_with_svd(X_scaled, n_components=1)

In [19]:
print('Норма Фробениуса:')
print(f' - исходной матрицы X:   {fro_norm(X_scaled)}')
print(f' - четырех ГК матрицы X: {fro_norm(X_pca_4)}')
print(f' - трех ГК матрицы X:    {fro_norm(X_pca_3)}')
print(f' - двух ГК матрицы X:    {fro_norm(X_pca_2)}')
print(f' - одной ГК матрицы X:   {fro_norm(X_pca_1)}')

Норма Фробениуса:
 - исходной матрицы X:   24.413111231467408
 - четырех ГК матрицы X: 24.41311123146741
 - трех ГК матрицы X:    24.34981497613762
 - двух ГК матрицы X:    23.896583749816834
 - одной ГК матрицы X:   20.853205381026378


Норма Фробениуса существенно изменяется с понижением количества главных компонент.

#### 3. Курсовая работа в папке course_work

Еще раз огрромное спасибо за курс)